In [1]:
porth ot333

NameError: name 'ot333' is not defined

<a href="https://colab.research.google.com/github/DarkPovoGang/DeepRL/blob/main/DeepRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Project

## Dataset preparation

In [ ]:
# !pip install torch ftfy regex tqdm
# !pip install git+https://github.com/openai/CLIP.git    

In [ ]:
import numpy as np
import torch 
import clip
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Torch version:", torch.__version__)
model, preprocess = clip.load("RN50")
model.eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

preprocess

In [ ]:
# dataset = RefCOCOg('.', 'val')
# for i in range(1):
#   x = dataset[i]
#   print(x)

## Our approach

In [2]:
# !apt-get install -y xvfb python-opengl swig x11-utils
# !pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

In [3]:
# from enum import Enum
# # class syntax
# class Actions(Enum):
#   ACT_RT = 0 #Right
#   ACT_LT = 1 #Left
#   ACT_UP = 2 #Up
#   ACT_DN = 3 #Down
#   ACT_TA = 4 #Taller
#   ACT_FA = 5 #Fatter
#   ACT_SR = 6 #Shorter
#   ACT_TH = 7 #Thiner
#   ACT_TR = 8 #Trigger


In [4]:
from custom_env import *

## Registring Environment

In [5]:
from gym.envs.registration import register
import gym

In [6]:
env_dict = gym.envs.registration.registry.env_specs.copy()
for env in env_dict:
    if 'DeepLearningProject/VisualGrounding-v0' in env:
        print("Remove {} from registry".format(env))
        del gym.envs.registration.registry.env_specs[env]
register(
    id='DeepLearningProject/VisualGrounding-v0', 
    entry_point='custom_env:VisualGroundingEnv',
    max_episode_steps=300
)

env = gym.make('DeepLearningProject/VisualGrounding-v0', split='val')
print("Added {} inside registry".format(env))

Dataset already extracted
Image shape: torch.Size([1, 1024]), Text shape: torch.Size([2, 1024])
Output shape: torch.Size([2, 1024])
Added <TimeLimit<VisualGroundingEnv<DeepLearningProject/VisualGrounding-v0>>> inside registry


### Network, Agent and Utils

In [7]:
import time
import collections

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import gym
import gym.spaces

from torch.utils.tensorboard import SummaryWriter

In [8]:
#TODO: change

class DQN(nn.Module):
    def __init__(self, input_features, n_actions, features=24):
        super(DQN, self).__init__()

        # multi layer perceptron
        self.mlp = nn.Sequential(
            nn.Linear(input_features, features),
            nn.ReLU(),
            nn.Linear(features, features * 2),
            nn.ReLU(),
            nn.Linear(features * 2, features * 4),
            nn.ReLU(),
            nn.Linear(features * 4, features * 2),
            nn.ReLU(),
            nn.Linear(features * 2, features),
            nn.ReLU(),
            nn.Linear(features, n_actions)
        )
    def forward(self, x):
        return self.mlp(x)

In [9]:
# create a subclass of Tuple with named attributes representing experience
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])

class ExperienceBuffer:
    def __init__(self, capacity):

        # represent the buffer as a deque
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):

        # add the current experience to the buffer
        self.buffer.append(experience)

    def sample(self, batch_size):

        # sample an index for each element in the batch
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)

        # extract experience entries for each element in the batch
        # each value returned by zip is a list of length batch_size
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        
        # print("REW:",rewards)
        
        # return results as numpy arrays
        return np.array(states), \
               np.array(actions), \
               np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), \
               np.array(next_states)

In [10]:
class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer

        self._reset()

    def _reset(self):
        # restarts the environment and reset the accumulated reward
        self.state, self.info = self.env.reset()
        self.total_reward = 0.0

    def play_step(self, net, epsilon=0.0, device="cpu"):

        # no need to create a computational graph when gathering experience
        with torch.no_grad():

            # will contain the total reward for the episode if the episode ends
            # or None otherwise
            done_reward = None

            # sample the action randomly with probability epsilon
            if np.random.random() < epsilon:
                action = self.env.action_space.sample()

            # otherwise, select action based on qvalues
            else:

                # creates a batch made of a single state
                # state_a = np.array([self.state], copy=False)
                state_tensor = torch.tensor(self.state['agent']).unsqueeze(0).to(device)

                # get qvalues and select the index of the maximum
                q_values = net(state_tensor)
                _, action_tensor = torch.max(q_values, dim=2)
                action = int(action_tensor.item())
                


            # perform a step in the environment
            new_state, reward, is_done, _ = self.env.step(action)
            # convert to toarch float
            new_state['agent'] = torch.tensor(new_state['agent']).clone().detach().float()
            self.total_reward += reward

            # save the new experience
            exp = Experience(self.state, action, reward, is_done, new_state['agent'])
            self.exp_buffer.append(exp)

            # registers the current state
            self.state['agent'] = new_state['agent']

            # Gets the current representation of the environment
            # current_rgb_image = self.env.render(mode='rgb_array')

            # if the episode is finished, reset the environment
            if is_done:
                done_reward = self.total_reward
                self._reset()

            return done_reward #, current_rgb_image

### Train Loop and Utils

In [11]:
DEFAULT_ENV_NAME = "DeepLearningProject/VisualGrounding-v0"

# we terminate training if the model on average balances the pole
# for at least 195 steps
MEAN_REWARD_BOUND = 195

# discount factor
GAMMA = 0.99

BATCH_SIZE = 32

# size of the replay buffer
REPLAY_SIZE = 10000

# warmup frames for the replay buffer
REPLAY_START_SIZE = 10000

LEARNING_RATE = 1e-4

# frequency for transferring weights from the actor DQN to the target DQN
SYNC_TARGET_FRAMES = 1000

# epsilon
EPSILON_DECAY_LAST_FRAME = 15000
EPSILON_START = 1.0
EPSILON_FINAL = 0.01

def train(net, target_net, env, buffer, agent, device, writer):

    # epsilon starts from the initial value and is then annealed
    epsilon = EPSILON_START

    # instantiate the optimizer. Note that target_net is not optimized
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

    total_rewards = []
    frame_idx = 0

    # frame idx and time at which the last episode ended
    ts_frame = 0
    ts = time.time()
    best_m_reward = None

    while True:
        frame_idx += 1

        # compute the current epsilon with linear annealing
        epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx / EPSILON_DECAY_LAST_FRAME)

        # perform a step in the environment to gather experience
        # reward, rbg_image = agent.play_step(net, epsilon, device=device)
        reward = agent.play_step(net, epsilon, device=device)

        # if the current episode has ended
        if reward is not None:

            # register the current total reward
            total_rewards.append(reward)

            # compute training speed
            speed = (frame_idx - ts_frame) / (time.time() - ts)
            ts_frame = frame_idx
            ts = time.time()

            # compute the mean reward over the last 100 episodes
            # print("tot_ew",total_rewards[-100:])
            m_reward = np.mean(total_rewards[-100:])
            # print("%d: done %d games, reward %.3f, eps %.2f, speed %.2f f/s" % (
            #     frame_idx, len(total_rewards), m_reward, epsilon, speed
            # ))

            # log values
            writer.add_scalar("epsilon", epsilon, frame_idx)
            writer.add_scalar("speed", speed, frame_idx)
            writer.add_scalar("reward_100", m_reward, frame_idx)
            writer.add_scalar("reward", reward, frame_idx)

            # update best rewards
            if best_m_reward is None or best_m_reward < m_reward:
                #torch.save(net.state_dict(), args.env + "-best_%.0f.dat" % m_reward)
                if best_m_reward is not None:
                    print("Best reward updated %.3f -> %.3f" % (best_m_reward, m_reward))
                best_m_reward = m_reward

            # stop training when a certain reward is achieved
            if m_reward > MEAN_REWARD_BOUND:
                print("Solved in %d frames!" % frame_idx)
                break

        # continue to collect experience until the warmup finishes
        if len(buffer) < REPLAY_START_SIZE:
            continue

        # at regular intervals load the weights of the actor DQN into the target DQN
        if frame_idx % SYNC_TARGET_FRAMES == 0:
            target_net.load_state_dict(net.state_dict())

        # perform an optimization step
        optimizer.zero_grad()
        batch = buffer.sample(BATCH_SIZE)
        loss_t = calc_loss(batch, net, target_net, device=device)
        loss_t.backward()
        optimizer.step()

In [12]:
def save_episode(net, env, agent, device, writer):

    from IPython import display as ipythondisplay
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

    # reset the environment
    agent._reset()
    all_frames = []

    reward = None

    # play an episode
    while reward is None:

        reward, rgb_image = agent.play_step(net, epsilon=0.0, device=device)

        # change fromat from (H, W, C) to (C, H, W) and saves the image
        rgb_image = torch.from_numpy(np.copy(rgb_image)).permute(2, 0, 1)
        all_frames.append(rgb_image)

    # video must be put into (batch, time, C, H, W) format to be saved
    video = torch.stack(all_frames, dim=0).unsqueeze(0)
    # save the video
    writer.add_video("sample_episode", video, global_step=0, fps=10)

In [13]:
def main():

    device = torch.device("cpu")

    # build the environment
    env = gym.make(DEFAULT_ENV_NAME, split="val")

    # create actor and target DQN models
    net = DQN(env.observation_space['agent'].shape[0], env.action_space.n).to(device)
    target_net = DQN(env.observation_space['agent'].shape[0], env.action_space.n).to(device)

    # net = DQN(len(env.observation_space), env.action_space.n).to(device)
    # target_net = DQN(env.observation_space.shape[0], env.action_space.n).to(device)

    # initialize the logger
    writer = SummaryWriter(log_dir="runs")

    # instantiate the experience buffer and the agent that collects experience
    buffer = ExperienceBuffer(REPLAY_SIZE)
    agent = Agent(env, buffer)

    # train the network
    train(net, target_net, env, buffer, agent, device, writer)

    # save a sample episode
    # save_episode(net, env, agent, device, writer)

    writer.close()


In [14]:
!rm -r runs

In [15]:
# %load_ext tensorboard
# %reload_ext tensorboard
# %tensorboard --logdir=runs

In [18]:
def calc_loss(batch, net, target_net, device="cpu"):

    # unpack the batch
    states, actions, rewards, dones, next_states = batch
    
    
    # WARNING: fix non verificato
    l = []
    for dict in states:
        l.append(dict["agent"])
    
    # states_v = torch.tensor(np.array(states, copy=False)).to(device)
    states_v = torch.tensor(np.array(l)).to(device)
    next_states_v = torch.tensor(np.array(next_states, copy=False)).to(device)
    # print("states" ,states_v)
    # print("next" ,next_states_v)
    # transform the batch elements to tensors

    
    
    # states_v = torch.from_numpy(states).to(device)
    # states_v = states[:]["agent"]
    # next_states_v = torch.from_numpy(next_states).to(device)
    
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.BoolTensor(dones).to(device)

    # infer the qvalues for the states
    state_qvalues = net(states_v)

    
    print("size before ",state_qvalues.shape)
    # extract the qvalues for the action that was selected
    #WARNING : SQUEEZE OF state_q_values and gather dim =0 
    state_qvalues = state_qvalues.squeeze()

    state_action_qvalues = state_qvalues.gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
    print("state qvalue \n", state_qvalues)
    print("unsqueezed actions \n", actions_v.unsqueeze(-1))
    print("result: ",state_action_qvalues)
    with torch.no_grad():
 
        # compute the qvalues for the next states using the target DQN
        next_state_qvalues = target_net(next_states_v)

        # extract the maximum one
        next_state_max_qvalue = next_state_qvalues.max(dim=1)[0]

        # if the next state refers to an ended episode, it has no value
        next_state_max_qvalue[done_mask] = 0.0

        next_state_max_qvalue = next_state_max_qvalue.detach()

    # Computes the expected qvalue using the Bellman equation
    # print(rewards_v.shape)
    # print(next_state_max_qvalue.shape)
    expected_state_action_qvalues = rewards_v + GAMMA * next_state_max_qvalue

    # Penalizes the DQN for inferring a qvalue different from the one
    # computed with the target DQN using the Bellman equation
    return nn.MSELoss()(state_action_qvalues, expected_state_action_qvalues)

In [19]:
main()

Dataset already extracted
Image shape: torch.Size([1, 1024]), Text shape: torch.Size([2, 1024])
Output shape: torch.Size([2, 1024])
Image shape: torch.Size([1, 1024]), Text shape: torch.Size([2, 1024])
Output shape: torch.Size([2, 1024])


/tmp/ipykernel_120796/2239051718.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_state['agent'] = torch.tensor(new_state['agent']).clone().detach().float()
/tmp/ipykernel_120796/2239051718.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state_tensor = torch.tensor(self.state['agent']).unsqueeze(0).to(device)


Image shape: torch.Size([1, 1024]), Text shape: torch.Size([2, 1024])
Output shape: torch.Size([2, 1024])
Image shape: torch.Size([1, 1024]), Text shape: torch.Size([2, 1024])
Output shape: torch.Size([2, 1024])
Best reward updated -47.871 -> -46.142
Image shape: torch.Size([1, 1024]), Text shape: torch.Size([1, 1024])
Output shape: torch.Size([1, 1024])
Best reward updated -46.142 -> -38.390
Image shape: torch.Size([1, 1024]), Text shape: torch.Size([1, 1024])
Output shape: torch.Size([1, 1024])
Image shape: torch.Size([1, 1024]), Text shape: torch.Size([2, 1024])
Output shape: torch.Size([2, 1024])
Image shape: torch.Size([1, 1024]), Text shape: torch.Size([2, 1024])
Output shape: torch.Size([2, 1024])
Image shape: torch.Size([1, 1024]), Text shape: torch.Size([2, 1024])
Output shape: torch.Size([2, 1024])
Image shape: torch.Size([1, 1024]), Text shape: torch.Size([2, 1024])
Output shape: torch.Size([2, 1024])
Image shape: torch.Size([1, 1024]), Text shape: torch.Size([2, 1024])
Outp

RuntimeError: The size of tensor a (32) must match the size of tensor b (9) at non-singleton dimension 1

In [ ]:
# CHECK REWARD VALUE

In [ ]:
# plt.figure(figsize=(10, 10))
# plt.subplot(2, 2, 1)
# plt.plot(logs["reward"])
# plt.title("training rewards (average)")
# plt.subplot(2, 2, 2)
# plt.plot(logs["step_count"])
# plt.title("Max step count (training)")
# plt.subplot(2, 2, 3)
# plt.plot(logs["eval reward (sum)"])
# plt.title("Return (test)")
# plt.subplot(2, 2, 4)
# plt.plot(logs["eval step_count"])
# plt.title("Max step count (test)")
# plt.show()

In [ ]:
x = np.array([{"a":2},{"b":23}])
# for i in range(x.size[0]):
#     print(x[i]["a"])
# # 

from torch import tensor

states= np.array([{'agent': tensor([[896.,   0., 639., 271.]]), 'target': tensor([[305.9500,  49.4000, 435.6600, 203.5200]])},
 {'agent': tensor([[377.,   0., 477., 576.]]), 'target': tensor([[  0.0000,   0.0000, 128.0000, 152.4500]])},
 {'agent': tensor([[426.,   0., 477., 512.]]), 'target': tensor([[259.1300, 120.0000, 336.0200, 363.2300]])},
 {'agent': tensor([[850., 150., 499., 374.]]), 'target': tensor([[270.5100,  19.3800, 410.3900, 168.5400]])},
 {'agent': tensor([[253.,   0., 424., 639.]]), 'target': tensor([[ 86.2600, 112.8500, 259.1900, 235.8500]])},
 {'agent': tensor([[426.,   0., 477., 512.]]), 'target': tensor([[259.1300, 120.0000, 336.0200, 363.2300]])},
 {'agent': tensor([[ 64.,   0., 639., 432.]]), 'target': tensor([[146.7000, 102.5400, 501.5700, 280.5200]])},
 {'agent': tensor([[832.,   0., 639., 479.]]), 'target': tensor([[378.0300,   4.7400, 640.0000, 256.1700]])},
 {'agent': tensor([[253.,   0., 424., 639.]]), 'target': tensor([[ 86.2600, 112.8500, 259.1900, 235.8500]])},
 {'agent': tensor([[ 88.,   0., 443., 639.]]), 'target': tensor([[ 43.2200,  78.1300, 204.4700, 623.3800]])} ])


y = torch.empty
l = []
for dict in states:
    l.append(dict["agent"])
print(l)

In [21]:
from torch import tensor
t = tensor([1,0])
print(t)
t = t.unsqueeze(-1)
print(t)
x = tensor([[1,2,3],
             [4,5,6]])
Y = x[0].gather(1, t)
print(Y)

tensor([1, 0])
tensor([[1],
        [0]])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)